In [1]:
! pip install gluonts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/NLP_project/chronos-forecasting')

In [5]:
import pandas as pd
import numpy as np
from gluonts.dataset.arrow import ArrowWriter

df = pd.read_csv("./data/btc_15m.csv")
df.drop_duplicates(inplace=True)

for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)

# Convert 'datetime' to proper datetime format
df["datetime"] = pd.to_datetime(df["datetime"])

# Ensure data is sorted by datetime (important for time series processing)
df = df.sort_values(by="datetime")

train_df = df[(df["datetime"] >= "2018-01-01") & (df["datetime"] < "2021-01-01")]
test_df = df[(df["datetime"] >= "2021-01-01") & (df["datetime"] <= "2022-12-31")]

In [6]:
df.head()

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13556.15,123.616013
1,2018-01-01 05:45:00,13533.75,13550.87,13402.00,13521.12,98.136430
2,2018-01-01 06:00:00,13500.00,13545.37,13450.00,13470.41,79.904037
3,2018-01-01 06:15:00,13494.65,13690.87,13450.00,13529.01,141.699719
4,2018-01-01 06:30:00,13528.99,13571.74,13402.28,13445.63,72.537533


In [7]:
print(train_df.count())

datetime    104732
open        104732
high        104732
low         104732
close       104732
volume      104732
dtype: int64


In [8]:
print(test_df.count())

datetime    37878
open        37878
high        37878
low         37878
close       37878
volume      37878
dtype: int64


In [9]:
from pathlib import Path

def convert_to_arrow(df, path):
    time_series = [
        {"start": df["datetime"].iloc[0], "target": df['close'].values}
    ]
    ArrowWriter(compression="lz4").write_to_file(time_series, path=path)


# Convert and save training data
convert_to_arrow(train_df, Path("./data/bitcoin-close-train.arrow"))

# Convert and save testing data
convert_to_arrow(test_df, Path("./data/bitcoin-close-test.arrow"))